In [1]:
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

In [2]:
# default: Load the model on the available device(s)
model = Qwen3VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen3-VL-2B-Instruct", dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-2B-Instruct")

In [3]:
def choose_trajectory(image_path):
    messages = [
        {"role": "system", "content": [
            {"type": "text", "text": "You are a helpful and precise assistant for car trajectory evaluation."}
            ]},
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image_path,
                },
                {"type": "text", "text": "Reason and pick the safest trajectory (color) for a car in the above image."},
            ],
        }
    ]

    # Preparation for inference
    inputs = processor.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt"
    )
    inputs = inputs.to(model.device)

    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=1024)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    print(output_text[0])

In [ ]:
choose_trajectory("./candidates/pedestrians_confusing.jpg")

Based on the provided image, here is an analysis of the trajectories and the safest path for a car:

-   **Green Arrow (Left):** This trajectory shows a car moving from the left side of the image towards the center, following the path of the pedestrian. This path is relatively clear and appears to be a direct, predictable route. However, it's not the most optimal path as it may be obstructed by the car in the foreground on the left.

-   **Red Arrow (Center):** This trajectory indicates a car moving from the left towards the center, but it is positioned in a way that it could be in the path of the pedestrian. This path is less safe because it could potentially cause a collision or require the car to make a sharp turn, which is not ideal.

-   **Blue Arrow (Right):** This trajectory shows a car moving from the right side of the image towards the center. This path is the most direct and safest, as it avoids the pedestrian and is not obstructed by any other vehicles. It is also the most p

In [4]:
choose_trajectory("./candidates/pedestrians.jpg")

Based on the image provided, here is an analysis of the trajectories:

- **Blue trajectory:** This path shows the car moving from the left side of the image, heading towards the center of the scene. It appears to be a direct, straight-line path through the open area of the parking lot, which is a common and safe route for a vehicle.

- **Red trajectory:** This path indicates the car moving from the left side of the image, turning to the right and heading towards the right side of the scene. This trajectory is less safe because it involves a sharp turn, which can be difficult for a driver to control, especially in a crowded or busy environment.

- **Green trajectory:** This path shows the car moving from the left side of the image, turning to the right and heading towards the right side of the scene. This trajectory is also less safe due to the sharp turn.

The safest trajectory for a car in the image is the blue one, as it is a direct path with no sharp turns or obstacles. The red and 